In [1]:
# FloodNet Data Exploration & Preprocessing
# **Objectives:**
# 1. Explore dataset structure and statistics
# 2. Analyze class distribution
# 3. Visualize sample images and masks
# 4. Set up preprocessing pipeline
# 5. Calculate class weights for handling imbalance

In [2]:
 ## 1. Setup and Imports

# %%
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


In [3]:
# Add src to path
sys.path.append('../src')

In [5]:
# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print(" Libraries imported successfully!")

 Libraries imported successfully!


In [ ]:
# 2. Configuration
# If config.py exists, import it; otherwise define inline
try:
    from utils.config import Config
    print(" Config loaded from file")
except ImportError:
    print(" Config not found, using inline configuration")
    
    class Config:
        # Update this path to match your setup
        BASE_DIR = Path(r"D:\Projects\Image Segmentation for Disaster Resilience\Disaster-segmentation")
        
        # Data paths
        RAW_DATA = BASE_DIR / "data" / "raw" / "FloodNet"
        TRAIN_IMAGES = RAW_DATA / "train" / "train-org-img"
        TRAIN_MASKS = RAW_DATA / "train" / "train-label-img"
        VAL_IMAGES = RAW_DATA / "val" / "val-org-img"
        VAL_MASKS = RAW_DATA / "val" / "val-label-img"
        TEST_IMAGES = RAW_DATA / "test" / "test-org-img"
        TEST_MASKS = RAW_DATA / "test" / "test-label-img"
        
        # Results
        RESULTS_DIR = BASE_DIR / "results"
        VIZ_DIR = RESULTS_DIR / "visualizations" / "data_exploration"
        
        # Parameters
        IMG_HEIGHT = 256
        IMG_WIDTH = 256
        NUM_CLASSES = 10
        
        # Classes
        CLASSES = {
            0: "Background", 1: "Building-flooded", 2: "Building-non-flooded",
            3: "Road-flooded", 4: "Road-non-flooded", 5: "Water",
            6: "Tree", 7: "Vehicle", 8: "Pool", 9: "Grass"
        }
        
        CLASS_COLORS = {
            0: (0, 0, 0), 1: (255, 0, 0), 2: (0, 0, 255),
            3: (255, 165, 0), 4: (128, 128, 128), 5: (0, 255, 255),
            6: (0, 255, 0), 7: (255, 0, 255), 8: (255, 255, 255), 9: (0, 128, 0)
        }

# Create visualization directory
Config.VIZ_DIR.mkdir(parents=True, exist_ok=True)
print(f"📂 Base Directory: {Config.BASE_DIR}")